#18_API를 활용한 크롤링

### 기초 개념
API란? 
- Application : 응용
- Program : 프로그램
- Interface : 연결고리 (inter(상호간의) + face(면))

예를 들어,  
내가 맛집 이름을 검색하면 네이버 지도 상에 맛집의 위치를 알려주는 기능을 구현하고 싶다.  
그럼, 네이버 지도 응용 프로그램을 사용해야겠죠?  
그런 응용 프로그램을 사용할 수 있게 연결해주는 연결 고리가 바로  
바로!!  
응용 프로그램의 연.결.고.리..!  
API입니다..

웹 API
웹 상의 그런 응용프로그램을 사용할 수 있게 해주는게 바로!!
웹 API입니다.  
  
  위에서 예를 든 네이버 지도 API도 당연히 웹 API겠죠
  
웹 API는 다음과 같이 2가지로 나눌 수 있답니다.
- REST API
- Streaming API 
  
  
한편, 이렇게 분류해볼 수도 있습니다.  
- API KEY가 필요한 경우
- 필요없는 경우

### 공공데이터포털 OPEN API

#### 가. 사전절차
1) 회원가입 후, 로그인  
2) API KEY 신청  
3) 원하는 API 검색 후, 활용 신청  
- 자동승인 : 바로 사용가능
- 심의승인 : 며칠 기달려야 함


#### 나. 사후절차
1) API KEY 디코딩 : utils.unquote (ㅠ.ㅠ)   
2) 필요한 파라미터 확인해서 query 구성 : 사이트에 명시  
3) requests.get  
- xml 경우, soup.select 또는 xmltodict.parse  
- json 경우, json.loads

In [0]:
import requests

# NOT REGISTERED ERROR 를 방지하기 위해 키를 디코딩해줘야 합니다.
# <SERVICE KEY IS NOT REGISTERED ERROR> 
My_Key = 'qJ6t5D6riLOPrFXladC2%2Fn0SNoL3H7LwwPVEa%2B1Qs%2FOQHoc6MQYZOd0XXksDMrnoCQx77QrXAcLR1a7u9tBwBQ%3D%3D'
decoded_Key = requests.utils.unquote(My_Key)

In [5]:
api_url = 'http://openapi.airkorea.or.kr/openapi/services/rest/UlfptcaAlarmInqireSvc/getUlfptcaAlarmInfo'

# 서비스키	ServiceKey	4	필수	-	공공데이터포털에서 받은 인증키
# 페이지 번호	pageNo	4	옵션	1	페이지번호
# 한 페이지 결과 수	numOfRows	4	옵션	10	한 페이지 결과 수
# 연도	year	4	필수	2013	측정 연도
# 항목명	itemCode	4	옵션	PM10	미세먼지 항목 구분(PM10, PM25) PM10, PM25 모두 조회할 경우 파라미터 생략

params = {
    'ServiceKey' : decoded_Key,
    'pageNo' : 1,
    'year' : 2020
}

resp = requests.get(api_url, params)
print(resp.text)

<?xml version="1.0" encoding="UTF-8"?>




<response>
	<header>
		<resultCode>00</resultCode>
		<resultMsg>NORMAL SERVICE.</resultMsg>
	</header>
	<body>
		<items>
			
				<item>
					<dataDate>2020-06-08</dataDate>
					<itemCode>PM25</itemCode>
					<districtName>전북</districtName>
					<moveName>완주권역</moveName>
					<issueDate>2020-06-08</issueDate>
					<issueTime>22:00</issueTime>
					<issueVal>106</issueVal>
					<issueGbn>주의보</issueGbn>
					<clearDate>--</clearDate>
					<clearTime>:00</clearTime>
					<clearVal></clearVal>
				</item>
			
				<item>
					<dataDate>2020-06-06</dataDate>
					<itemCode>PM25</itemCode>
					<districtName>전북</districtName>
					<moveName>김제권역</moveName>
					<issueDate>2020-06-06</issueDate>
					<issueTime>15:00</issueTime>
					<issueVal>89</issueVal>
					<issueGbn>주의보</issueGbn>
					<clearDate>2020-06-06</clearDate>
					<clearTime>18:00</clearTime>
					<clearVal>29</clearVal>
				</item>
			
				<item>
					<dataDate>2020-06-06</dataDate>
	

In [6]:
# soup으로 파싱한다.
from bs4 import BeautifulSoup

soup = BeautifulSoup(resp.text, 'lxml')
data = soup.select('item cleardate')
data

[<cleardate>--</cleardate>,
 <cleardate>2020-06-06</cleardate>,
 <cleardate>2020-06-06</cleardate>,
 <cleardate>2020-06-06</cleardate>,
 <cleardate>2020-06-05</cleardate>,
 <cleardate>2020-06-05</cleardate>,
 <cleardate>2020-06-06</cleardate>,
 <cleardate>2020-05-26</cleardate>,
 <cleardate>2020-05-15</cleardate>,
 <cleardate>2020-05-15</cleardate>]

In [0]:
# pip install xmltodict

In [8]:
#xmltodict 를 이용해 dictionary로 변환한 뒤, dictionary로 처리한다.
import xmltodict

data = xmltodict.parse(resp.text)
data

OrderedDict([('response',
              OrderedDict([('header',
                            OrderedDict([('resultCode', '00'),
                                         ('resultMsg', 'NORMAL SERVICE.')])),
                           ('body',
                            OrderedDict([('items',
                                          OrderedDict([('item',
                                                        [OrderedDict([('dataDate',
                                                                       '2020-06-08'),
                                                                      ('itemCode',
                                                                       'PM25'),
                                                                      ('districtName',
                                                                       '전북'),
                                                                      ('moveName',
                                                                

In [9]:
itemList = data['response']['body']['items']['item']

for item in itemList:
  print(item['clearDate'])

--
2020-06-06
2020-06-06
2020-06-06
2020-06-05
2020-06-05
2020-06-06
2020-05-26
2020-05-15
2020-05-15


### KAKAO API

#### 가. 사전절차
- https://developers.kakao.com/console/app 로그인
- 내 애플리케이션 만들기



In [10]:
url='https://dapi.kakao.com/v2/local/search/keyword.json'

params = {
    'query' : '구리역',
    'page' : 1,
    'size' : 15
}

headers={'Authorization':'KakaoAK fa741b81478dd68989960b52a562d96f'}
result=requests.get(url,params=params, headers=headers).json()
result

{'documents': [{'address_name': '경기 구리시 인창동 244-2',
   'category_group_code': 'SW8',
   'category_group_name': '지하철역',
   'category_name': '교통,수송 > 지하철,전철 > 경의중앙선',
   'distance': '',
   'id': '21160922',
   'phone': '031-552-7784',
   'place_name': '구리역 경의중앙선',
   'place_url': 'http://place.map.kakao.com/21160922',
   'road_address_name': '경기 구리시 건원대로34번길 32-29',
   'x': '127.143350984187',
   'y': '37.6032707405088'},
  {'address_name': '경기 구리시 인창동 316-58',
   'category_group_code': 'PK6',
   'category_group_name': '주차장',
   'category_name': '교통,수송 > 교통시설 > 주차장',
   'distance': '',
   'id': '20513666',
   'phone': '',
   'place_name': '구리역 주차장',
   'place_url': 'http://place.map.kakao.com/20513666',
   'road_address_name': '',
   'x': '127.141683540013',
   'y': '37.6030240809108'},
  {'address_name': '경기 구리시 인창동 676-6',
   'category_group_code': '',
   'category_group_name': '',
   'category_name': '가정,생활 > 생활용품점 > 다이소',
   'distance': '',
   'id': '12965637',
   'phone': '031-556-6